## NB
-  The website to get the dataset for this course is [here](https://nyc.gov/site/tlc/about/tlc-trip-record-data.page)
- Find Microsoft's article on MLOPS-Maturity Model [here](https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/mlops-maturity-model)
- To connect to mlflow via terminal : mlflow ui --backend-store-uri sqlite:///mlflow.db

In [1]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/Week02-Experiment-Tracking/mlruns/2', creation_time=1716380685365, experiment_id='2', last_update_time=1716380685365, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
pd.__version__

'2.1.4'

In [4]:
#!pip install pyarrow

In [5]:
def readdataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime 
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[((df.duration >= 1) & (df.duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype('str')
    
    return df

In [6]:
df_train = readdataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = readdataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO']
    # 'PULocationID', 'DOLocationID']
numerical = ["trip_distance"]

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [11]:
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

7.758715210452687

In [ ]:
with mlflow.start_run():
    alpha = 0.001
    mlflow.log_param("alpha", alpha)
    mlflow.set_tag("developer", "Mardiyyah")
    mlflow.log_param("train-data-path", "Week02-Experiment-Tracking/data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "Week02-Experiment-Tracking/data/green_tripdata_2021-02.parquet")

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared = False)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
lasso = Lasso()

lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

7.563048898788465

In [17]:
ridge = Ridge()

ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

7.703735185818828

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [14]:
# !pip3 install xgboost
# !pip3 install hyperopt

In [13]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [ ]:
params = {
   'learning_rate':0.09490529813764448,
    'max_depth': 42,
    'min_child_weight': 0.4339829535913446,
    'objective': 'reg:linear',
    'reg_alpha': 0.007243881506749969,
    'reg_lambda': 0.004147211487368328,
    'seed': 42}

mlflow.xgboost.autolog(disable=True) #autologging disabled
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

In [16]:
with mlflow.start_run():
    best_params = {
    'learning_rate':0.09490529813764448,
        'max_depth': 42,
        'min_child_weight': 0.4339829535913446,
        'objective': 'reg:linear',
        'reg_alpha': 0.007243881506749969,
        'reg_lambda': 0.004147211487368328,
        'seed': 42}
    
    mlflow.log_params(best_params)

    booster = xgb.train(
                params=best_params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:20:26] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.45718
[1]	validation-rmse:10.79706
[2]	validation-rmse:10.21623
[3]	validation-rmse:9.71139
[4]	validation-rmse:9.27225
[5]	validation-rmse:8.89528
[6]	validation-rmse:8.56448
[7]	validation-rmse:8.28646
[8]	validation-rmse:8.04477
[9]	validation-rmse:7.83726
[10]	validation-rmse:7.66186
[11]	validation-rmse:7.51076
[12]	validation-rmse:7.38220
[13]	validation-rmse:7.27473
[14]	validation-rmse:7.17678
[15]	validation-rmse:7.09777
[16]	validation-rmse:7.03003
[17]	validation-rmse:6.97139
[18]	validation-rmse:6.91942
[19]	validation-rmse:6.87668
[20]	validation-rmse:6.83876
[21]	validation-rmse:6.80208
[22]	validation-rmse:6.77339
[23]	validation-rmse:6.74882
[24]	validation-rmse:6.72698
[25]	validation-rmse:6.70742
[26]	validation-rmse:6.69134
[27]	validation-rmse:6.67518
[28]	validation-rmse:6.66305
[29]	validation-rmse:6.65299
[30]	validation-rmse:6.64275
[31]	validation-rmse:6.63414
[32]	validation-rmse:6.62597
[33]	validation-rmse:6.61894
[34]	validation-rmse:

/home/codespace/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:23:08] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [20]:
with open('models/lasso.bin', 'wb') as f_las:
    pickle.dump((dv, lasso), f_las)